## LSSTCam Astrometry learning

Craig Lage - 06-May-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.afw import cameraGeom
import lsst.geom
import lsst.geom as geom
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from os import listdir
import healpy as hp
from astropy.table import Table, vstack
from astropy.coordinates import angular_separation
import cv2


## The cell below is from Shuang.  I have modified it to use focal plane coordinates to allow applying the WCS from one detector to another detector

In [ ]:
def get_stars_focal_plane(detector,ra,dec,cam_wcs,npixedge=0):
    # some constants and useful objects
    ccd_diag = 0.15852  #Guider CCD diagonal radius in Degrees
    path = '/home/s/shuang92/rubin-user/Monster_guide'  ## RSP path
    res = 5
    nside = 2**res
    npix = 12 * nside**2
    bad_guideramps = {193: 'C1', 198: 'C1', 201: 'C0'}
    filters = ['u','g','r','i','z','y']
    
    # query the Monster
    hp_ind = hp.ang2pix(nside, ra, dec, lonlat=True)

    # should only need at most 4 tiles, but for simplicity using 9 Tables
    SW, W, NW, N, NE, E, SE, S = hp.get_all_neighbours(nside, hp_ind)
    
    this_table = Table.read(f'{path}/{hp_ind}.csv')

    E_table = Table.read(f'{path}/{E}.csv')
    W_table = Table.read(f'{path}/{W}.csv')
    S_table = Table.read(f'{path}/{S}.csv')
    N_table = Table.read(f'{path}/{N}.csv')

    SW_table = Table.read(f'{path}/{SW}.csv')
    SE_table = Table.read(f'{path}/{SE}.csv')
    NW_table = Table.read(f'{path}/{NW}.csv')
    NE_table = Table.read(f'{path}/{NE}.csv')

    star_cat = vstack([this_table, E_table, W_table, S_table, N_table, SW_table, SE_table, NW_table, NE_table])

    # find the ra,dec of the CCD center
    bbox = detector.getBBox()
    nx,ny = bbox.getDimensions()
    detName = detector.getName()
    lct = LsstCameraTransforms(camera,detName)
    llfpX, llfpY = lct.ccdPixelToFocalMm(0, 0, detName)
    llfpX = llfpX * 100 + int(nx/2) # Convert to pixels and recenter
    llfpY = llfpY * 100 + int(ny/2) # Convert to pixels and recenter
    urfpX, urfpY = lct.ccdPixelToFocalMm(nx, ny, detName)
    urfpX = urfpX * 100 + int(nx/2) # Convert to pixels and recenter
    urfpY = urfpY * 100 + int(ny/2) # Convert to pixels and recenter
    if llfpX > urfpX:
        llfpX, urfpX = urfpX, llfpX
    if llfpY > urfpY:
        llfpY, urfpY = urfpY, llfpY
    #print(detName, llfpX, llfpY, urfpX, urfpY)        
    x0 = int((urfpX + llfpX) / 2) # In pixels
    y0 = int((urfpY + llfpY) / 2) # In pixels
    ra_ccd,dec_ccd = cam_wcs.pixelToSky(x0,y0)

    ccd_bbox = Box2I(Point2I(llfpX, llfpY), 
                     Point2I(urfpX, urfpY))
    star_cat['dangle'] = np.degrees(angular_separation(ra_ccd.asRadians(),dec_ccd.asRadians(),
                            star_cat['coord_ra'],star_cat['coord_dec']))
    insideCCDradius = (star_cat['dangle']<ccd_diag) # inside the Guider CCD radius

    # Selection 1: the Star is isolated and inside the CCD radius
    cat_select1 = star_cat[(insideCCDradius)]

    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = cam_wcs.skyToPixelArray(cat_select1['coord_ra'],cat_select1['coord_dec'],degrees=False)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
    cat_select1['ccdx'] = ccdx - llfpX
    cat_select1['ccdy'] = ccdy - llfpY
    cat_select1['inCCD'] = inCCD

    return cat_select1

ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }

## This cell builds a blank guider CCD image with amps marked, and then adds the stamp.
## The final image is in focal plane orientation.

In [ ]:
def make_guider_image(detector, raw):
    detName = detector.getName()
    det_bbox = detector.getBBox()
    lct = LsstCameraTransforms(camera,detector.getName())
    background = 20
    dummy = np.zeros([det_bbox.endX, det_bbox.endX]) + background
    print(dummy.shape)
    # Make gridlines
    line_level = 100
    colour = (255, 255, 255)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 3
    thickness = 10
    for amp in detector:
        ampName = amp.getName()
        amp_bbox = amp.getBBox()
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        #print(ampName, x0, x1, y0, y1)
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        #print(ampName, x0, x1, y0, y1)
        dummy[y0:y1, x0:x1] = line_level
        amp_bbox.grow(-4)
        x0, y0 = amp_bbox.beginX, amp_bbox.beginY
        x1, y1 = amp_bbox.endX, amp_bbox.endY
        #print(ampName, x0, x1, y0, y1)
        if x0 > x1:
            x0, x1 = x1, x0
        if y0 > y1:
            y0, y1 = y1, y0
        #print(ampName, x0, x1, y0, y1)
        dummy[y0:y1, x0:x1] = background
        origin = (int((x0+x1)/2)-80, y1-80)
        cv2.putText(dummy, ampName, origin, font, scale, colour, thickness, bottomLeftOrigin=True)

    # Add the stamp image
    mdata = raw.metadata
    s_llX = mdata['ROICOL']
    s_llY = mdata['ROIROW']
    dx = mdata['ROICOLS']
    dy = mdata['ROIROWS']
    stamp_ampName = ampDict[mdata['ROISEG']]
    print(s_llX, s_llY, dx, dy, stamp_ampName)
    x0, y0 = lct.ampPixelToCcdPixel(s_llX, s_llY, stamp_ampName, detName)
    print(x0, y0)
    x0 = int(x0)
    y0 = int(y0)
    x1 = x0 - dx
    y1 = y0 + dy
    if x0 > x1:
        x0, x1 = x1, x0
    if y0 > y1:
        y0, y1 = y1, y0

    print(x0, x1, y0, y1)
    arr = raw[2].stamp_im.image.array
    arr = arr - np.median(arr)
    print(arr.shape)
    flip_arr = np.fliplr(arr)
    print(flip_arr.shape)
    print(dummy[y0:y1, x0:x1].shape)
    dummy[y0:y1, x0:x1] = flip_arr
    k = detector.getOrientation().getNQuarter()
    dummy = np.rot90(dummy, k=k, axes=(1,0))
    return dummy

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157',
                                             "LSSTCam/raw/guider"])

## Now apply it to an image

In [ ]:
#expId = 2025042500591
#expId = 2025050200787
#expId = 2025050200769
expId = 2025050200713

camera = LsstCam.getCamera()

ref_raft = 'R22'
ref_ccd = 'S11'
ref_detName = f"{ref_raft}_{ref_ccd}"
for ref_detector in camera:
    if ref_detector.getName()== ref_detName:
        break
ref_calexp = butler.get('preliminary_visit_image', detector=ref_detector.getId(), visit=expId)
ref_wcs = ref_calexp.getWcs()
ref_bbox = ref_detector.getBBox()
ref_center = ref_wcs.pixelToSky(ref_bbox.centerX, ref_bbox.centerY)
ref_ra_center = ref_center.getRa().asDegrees()
ref_dec_center = ref_center.getDec().asDegrees()

img_raft = 'R13'
img_ccd = 'S00'
img_detName = f"{img_raft}_{img_ccd}"
for img_detector in camera:
    if img_detector.getName()== img_detName:
        break
img_calexp = butler.get('preliminary_visit_image', detector=img_detector.getId(), visit=expId)
img_wcs = img_calexp.getWcs()
img_bbox = img_detector.getBBox()
img_center = img_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
img_ra_center = img_center.getRa().asDegrees()
img_dec_center = img_center.getDec().asDegrees()

cat_select = get_stars_focal_plane(img_detector,img_ra_center,img_dec_center,ref_wcs)
xs = []
ys = []
mags = []
for i in range(len(cat_select)):
    x = cat_select['ccdx'][i]
    y = cat_select['ccdy'][i]
    if cat_select['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select['mag_r'][i]:.1f}")

# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(16,16))
plt.title(f"{expId} {img_raft}_{img_ccd}, WCS from {ref_raft}_{ref_ccd}")
img = plt.imshow(img_calexp.image.array,  interpolation='Nearest', cmap='gray', vmin=0, vmax=1000, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/WCS_{expId}_{img_raft}_{img_ccd}_WCS_From_{ref_raft}_{ref_ccd}.png")

## Now look at the guider CCDs

In [ ]:
img_raft = 'R00'
img_ccd = 'SG1'
img_detName = f"{img_raft}_{img_ccd}"
for img_detector in camera:
    if img_detector.getName()== img_detName:
        break
img_bbox = img_detector.getBBox()
img_center = ref_wcs.pixelToSky(img_bbox.centerX, img_bbox.centerY)
img_ra_center = img_center.getRa().asDegrees()
img_dec_center = img_center.getDec().asDegrees()

cat_select = get_stars_focal_plane(img_detector,img_ra_center,img_dec_center,ref_wcs)
xs = []
ys = []
mags = []
for i in range(len(cat_select)):
    x = cat_select['ccdx'][i]
    y = cat_select['ccdy'][i]
    if cat_select['inCCD'][i]:
        xs.append(x)
        ys.append(y)
        mags.append(f"{cat_select['mag_r'][i]:.1f}")

# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar
    
if expId > 2025050900000:
    raw_detName = img_detName
else:
    print("Swapping SG0, SG1")
    # This is necessary because SG0 and SG1 were swapped for a time
    if img_detName.split('_')[1] == 'SG0':
        raw_detName = img_detName.split('_')[0] + '_SG1'
    if img_detName.split('_')[1] == 'SG1':
        raw_detName = img_detName.split('_')[0] + '_SG0'
for raw_detector in camera:
    if raw_detector.getName()== raw_detName:
        break

raw = butler.get("guider_raw", exposure=expId, detector=raw_detector.getId())
dummy = make_guider_image(img_detector, raw)
plt.figure(figsize=(10,10))
plt.title(f"{expId} {img_raft}_{img_ccd}, WCS from {ref_raft}_{ref_ccd}", fontsize=24)
img = plt.imshow(dummy,  interpolation='Nearest', cmap='gray', vmin=0, vmax=100, origin='lower')
plt.scatter(xs, ys\
            ,facecolors='none', edgecolors='g', s=200, lw=2)
for x, y, mag in zip(xs, ys, mags):
    plt.text(x+50,y, mag, color='g')
colorbar(img)
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/WCS_{expId}_{img_raft}_{img_ccd}_WCS_From_{ref_raft}_{ref_ccd}_New.png")